In [1]:
import pandas as pd
from langdetect import detect
import unidecode
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [2]:
def convert_unidecode(x):
    return unidecode.unidecode(x)

In [3]:
# remove ponctuation
def remove_punct1(x):
    p = ['!', '(', ')', '-', '[', ']', '{', '}', ';', ':', "'", '"', '\\', ',', '<', '>', '.', '/', '?', '@', '#', '$', '%', '^', '&', '*', '_', '~']
    return ' '.join([string for string in x.split(' ') if string not in p])

In [4]:
# remove ponctuation
def remove_punct2(x):
    import re
    return re.sub(r'[^\w]|[^\w] ', ' ', x)

In [5]:
# Define stop words to clean text (job description)
stopwords_en = set(stopwords.words('english'))

stopwords_pt = set(stopwords.words('portuguese'))

stopwords_es = set(stopwords.words('spanish'))

stopwords_de = set(stopwords.words('german'))

stopwords_nl = set(stopwords.words('dutch'))

stopwords_fr = set(stopwords.words('french'))

stopwords_fi = set(stopwords.words('finnish'))

In [6]:
def remove_stop_words(x):
    return ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_en)]) if detect(x)=='en'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_pt)]) if detect(x)=='pt'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_es)]) if detect(x)=='es'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_de)]) if detect(x)=='de'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_fr)]) if detect(x)=='fr'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_nl)]) if detect(x)=='nl'\
    else ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in (stopwords_fi)])

In [7]:
def remove_stop_words1(x):
    stop_words_concat = list(stopwords_en)+list(stopwords_es)+list(stopwords_pt)
    return ' '.join([word.lower() for word in x.split() \
                   if word.lower() not in stop_words_concat])

In [8]:
def stem_snowball(x):
    if detect(x) == 'pt':
        stemmer = SnowballStemmer('portuguese')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'en':
        stemmer = SnowballStemmer('english')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'es':
        stemmer = SnowballStemmer('spanish')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'de':
        stemmer = SnowballStemmer('german')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'nl':
        stemmer = SnowballStemmer('dutch')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'fr':
        stemmer = SnowballStemmer('french')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    elif detect(x) == 'fi':
        stemmer = SnowballStemmer('finnish')
        tokens = nltk.word_tokenize(x)
        return [stemmer.stem(token) for token in tokens]
    else:
        return nltk.word_tokenize(x)

In [9]:
def bigrams(x):
    try:
        #Tokens = nltk.word_tokenize(x)
        output = list(nltk.bigrams(x))
        return output
    except:
        pass

In [10]:
def trigram(x):
    try:
        #Tokens = nltk.word_tokenize(x)
        output = list(nltk.trigrams(x))
        return output
    except:
        pass

In [11]:
def token(x):
    return nltk.word_tokenize(x)

In [12]:
df = pd.read_csv('/home/inrx/Ironhack/TA/job-search/linkedin-project/linkedin_search.csv')
print (df.shape)
df.head()

(723, 12)


,title,company,company-link,post-date,applications,location,role-description,link,"experience,",type_work,function,sector_final
0,"Data Analyst - Milano, Milano",P. & P. Consulting,https://it.linkedin.com/company/programs-&-pro...,2 days ago,Be among the first 25 applicants,"Milano, IT","Ricerchiamo un Data Analyst, per attività di a...",https://www.linkedin.com/jobs/view/1605083546,Entry level,Full-time,Information Technology,"['', 'Information ', 'Technology and ', 'Servi..."
1,Data Analyst,Cromwell Property Services,https://uk.linkedin.com/company/cromwell?trk=g...,3 weeks ago,Be among the first 25 applicants,"Leicester, GB",As we embark on an exciting period of change t...,https://www.linkedin.com/jobs/view/1564171840,Entry level,Full-time,Information Technology,"['', 'Staffing and ', 'Recruiting', 'Consumer ..."
2,Data Analyst Intern,The Shortcut,https://fi.linkedin.com/company/theshortcut?tr...,5 days ago,Be among the first 25 applicants,"Helsinki, FI","Are you a bit mad, very driven, and excited ab...",https://www.linkedin.com/jobs/view/1599766365,Internship,Internship,Information Technology,"['', 'Marketing and ', 'Advertising', 'Compute..."
3,Data Analyst (Business Intelligence) (m/w/d),atene KOM GmbH,https://de.linkedin.com/company/atene-kom-gmbh...,5 days ago,Be among the first 25 applicants,"Berlin, DE",Für unser Büro-Loft in Berlin-Friedrichshain s...,https://www.linkedin.com/jobs/view/1599163575,Associate,Full-time,Information Technology,"['', 'Information ', 'Technology and ', 'Servi..."
4,Data Insight Analyst - London,Metrica Recruitment,https://www.linkedin.com/company/metrica-recru...,4 weeks ago,Be among the first 25 applicants,"London, GB",**The Company**\nA leading provider of researc...,https://www.linkedin.com/jobs/view/1603033659,Entry level,Full-time,Other,"['', 'Information ', 'Technology and ', 'Servi..."


In [13]:
df = df.drop_duplicates(keep='last')
df = df.reset_index(drop=True)
df.shape

(522, 12)

In [14]:
df = df.replace(to_replace=r'\n', value=' ', regex=True)

In [15]:
df['role-description'] = df['role-description'].apply(remove_punct1)
df['role-description'] = df['role-description'].apply(remove_punct2)

In [16]:
df['role-description'] = df['role-description'].apply(lambda x: x.lower())

In [17]:
# Clean the text according remove the stopwords according to the language of the job post
df['role_description1'] = df['role-description'].apply(remove_stop_words)

# Clean the text according remove the stopwords regardless the language of the job post
df['role_description1'] = df['role_description1'].apply(remove_stop_words1)

In [18]:
df['role-description'] = df['role-description'].apply(convert_unidecode)

In [19]:
df['role_description1'] = df['role_description1'].apply(stem_snowball)

In [20]:
df['lang'] = df['role-description'].apply(lambda x: detect(x))

In [21]:
df = df[df['lang'].map(df['lang'].value_counts()) > 2]

In [22]:
df.head()

,title,company,company-link,post-date,applications,location,role-description,link,"experience,",type_work,function,sector_final,role_description1,lang
0,"Data Analyst - Milano, Milano",P. & P. Consulting,https://it.linkedin.com/company/programs-&-pro...,2 days ago,Be among the first 25 applicants,"Milano, IT",ricerchiamo un data analyst per attivita di a...,https://www.linkedin.com/jobs/view/1605083546,Entry level,Full-time,Information Technology,"['', 'Information ', 'Technology and ', 'Servi...","[ricerchiamo, data, analyst, per, attività, di...",it
1,Data Analyst,Cromwell Property Services,https://uk.linkedin.com/company/cromwell?trk=g...,3 weeks ago,Be among the first 25 applicants,"Leicester, GB",as we embark on an exciting period of change t...,https://www.linkedin.com/jobs/view/1564171840,Entry level,Full-time,Information Technology,"['', 'Staffing and ', 'Recruiting', 'Consumer ...","[embark, excit, period, chang, throughout, bus...",en
2,Data Analyst Intern,The Shortcut,https://fi.linkedin.com/company/theshortcut?tr...,5 days ago,Be among the first 25 applicants,"Helsinki, FI",are you a bit mad very driven and excited ab...,https://www.linkedin.com/jobs/view/1599766365,Internship,Internship,Information Technology,"['', 'Marketing and ', 'Advertising', 'Compute...","[bit, mad, driven, excit, build, new, thing, l...",en
3,Data Analyst (Business Intelligence) (m/w/d),atene KOM GmbH,https://de.linkedin.com/company/atene-kom-gmbh...,5 days ago,Be among the first 25 applicants,"Berlin, DE",fur unser buro loft in berlin friedrichshain s...,https://www.linkedin.com/jobs/view/1599163575,Associate,Full-time,Information Technology,"['', 'Information ', 'Technology and ', 'Servi...","[buro, loft, berlin, friedrichshain, such, ab,...",de
4,Data Insight Analyst - London,Metrica Recruitment,https://www.linkedin.com/company/metrica-recru...,4 weeks ago,Be among the first 25 applicants,"London, GB",the company a leading provider of research...,https://www.linkedin.com/jobs/view/1603033659,Entry level,Full-time,Other,"['', 'Information ', 'Technology and ', 'Servi...","[compani, lead, provid, research, data, strate...",en


In [23]:
df['role_des_bigrams'] = df['role_description1'].apply(bigrams)

In [24]:
df['role_des_trigrams'] = df['role_description1'].apply(trigram)

In [25]:
df.head()

,title,company,company-link,post-date,applications,location,role-description,link,"experience,",type_work,function,sector_final,role_description1,lang,role_des_bigrams,role_des_trigrams
0,"Data Analyst - Milano, Milano",P. & P. Consulting,https://it.linkedin.com/company/programs-&-pro...,2 days ago,Be among the first 25 applicants,"Milano, IT",ricerchiamo un data analyst per attivita di a...,https://www.linkedin.com/jobs/view/1605083546,Entry level,Full-time,Information Technology,"['', 'Information ', 'Technology and ', 'Servi...","[ricerchiamo, data, analyst, per, attività, di...",it,"[(ricerchiamo, data), (data, analyst), (analys...","[(ricerchiamo, data, analyst), (data, analyst,..."
1,Data Analyst,Cromwell Property Services,https://uk.linkedin.com/company/cromwell?trk=g...,3 weeks ago,Be among the first 25 applicants,"Leicester, GB",as we embark on an exciting period of change t...,https://www.linkedin.com/jobs/view/1564171840,Entry level,Full-time,Information Technology,"['', 'Staffing and ', 'Recruiting', 'Consumer ...","[embark, excit, period, chang, throughout, bus...",en,"[(embark, excit), (excit, period), (period, ch...","[(embark, excit, period), (excit, period, chan..."
2,Data Analyst Intern,The Shortcut,https://fi.linkedin.com/company/theshortcut?tr...,5 days ago,Be among the first 25 applicants,"Helsinki, FI",are you a bit mad very driven and excited ab...,https://www.linkedin.com/jobs/view/1599766365,Internship,Internship,Information Technology,"['', 'Marketing and ', 'Advertising', 'Compute...","[bit, mad, driven, excit, build, new, thing, l...",en,"[(bit, mad), (mad, driven), (driven, excit), (...","[(bit, mad, driven), (mad, driven, excit), (dr..."
3,Data Analyst (Business Intelligence) (m/w/d),atene KOM GmbH,https://de.linkedin.com/company/atene-kom-gmbh...,5 days ago,Be among the first 25 applicants,"Berlin, DE",fur unser buro loft in berlin friedrichshain s...,https://www.linkedin.com/jobs/view/1599163575,Associate,Full-time,Information Technology,"['', 'Information ', 'Technology and ', 'Servi...","[buro, loft, berlin, friedrichshain, such, ab,...",de,"[(buro, loft), (loft, berlin), (berlin, friedr...","[(buro, loft, berlin), (loft, berlin, friedric..."
4,Data Insight Analyst - London,Metrica Recruitment,https://www.linkedin.com/company/metrica-recru...,4 weeks ago,Be among the first 25 applicants,"London, GB",the company a leading provider of research...,https://www.linkedin.com/jobs/view/1603033659,Entry level,Full-time,Other,"['', 'Information ', 'Technology and ', 'Servi...","[compani, lead, provid, research, data, strate...",en,"[(compani, lead), (lead, provid), (provid, res...","[(compani, lead, provid), (lead, provid, resea..."


In [26]:
trigrams = {tup:lst.count(tup) for lst in df['role_des_trigrams'] for tup in lst}

trigrams = {k: v for k, v in trigrams.items() if v > 2}

trigrams_sorted = sorted(trigrams.items(), key=lambda kv: kv[1], reverse=True)

In [27]:
bigrams = {tup:lst.count(tup) for lst in df['role_des_bigrams'] for tup in lst}

bigrams = {k: v for k, v in bigrams.items() if v > 2}

bigrams_sorted = sorted(bigrams.items(), key=lambda kv: kv[1], reverse=True)

In [28]:
tokens = {tup:lst.count(tup) for lst in df['role_description1'] for tup in lst}

tokens = {k: v for k, v in tokens.items() if v > 5}

tokens_sorted = sorted(tokens.items(), key=lambda kv: kv[1], reverse=True)

In [29]:
bigrams_sorted

[(('group', 'intern'), 15),
 (('uk', 'biobank'), 11),
 (('market', 'fundament'), 10),
 (('heineken', 'netherland'), 9),
 (('ground', 'truth'), 9),
 (('h', 'b'), 9),
 (('big', 'dat'), 8),
 (('00', 'h'), 8),
 (('bnp', 'pariba'), 7),
 (('gobiern', 'dat'), 7),
 (('risk', 'data'), 7),
 (('nation', 'grid'), 7),
 (('porsch', 'consult'), 7),
 (('element', 'prototyp'), 7),
 (('transit', 'data'), 6),
 (('civil', 'servic'), 6),
 (('financi', 'crime'), 6),
 (('jam', 'citi'), 6),
 (('digit', 'social'), 6),
 (('small', 'busi'), 6),
 (('financi', 'well'), 6),
 (('stress', 'test'), 6),
 (('back', 'market'), 6),
 (('leadership', 'partner'), 6),
 (('goldman', 'sach'), 6),
 (('social', 'media'), 5),
 (('business', 'data'), 5),
 (('lead', 'instructor'), 5),
 (('mobil', 'game'), 5),
 (('visa', 'bank'), 5),
 (('azur', 'data'), 5),
 (('intelligenc', 'analyst'), 5),
 (('r', 'solut'), 5),
 (('virgin', 'media'), 5),
 (('custom', 'dat'), 5),
 (('mobilcom', 'debitel'), 5),
 (('market', 'model'), 5),
 (('fundament

In [30]:
trigrams_sorted

[(('group', 'intern', 'audit'), 14),
 (('data', 'analyt', 'offic'), 8),
 (('associ', 'data', 'analyst'), 8),
 (('credit', 'risk', 'data'), 7),
 (('risk', 'data', 'analyst'), 7),
 (('business', 'intelligenc', 'analyst'), 5),
 (('business', 'data', 'business'), 5),
 (('data', 'business', 'intelligenc'), 5),
 (('market', 'fundament', 'team'), 5),
 (('porsch', 'consult', 'inc'), 5),
 (('salesforc', 'salesforc', 'org'), 4),
 (('bi', 'power', 'app'), 4),
 (('career', 'nationalarch', 'gov'), 4),
 (('digit', 'social', 'media'), 4),
 (('sql', 'data', 'analyst'), 4),
 (('digit', 'platform', 'servic'), 4),
 (('custom', 'dat', 'analyst'), 4),
 (('aalbert', 'surfac', 'treatment'), 4),
 (('customer', 'data', 'analyst'), 4),
 (('high', 'qualiti', 'servic'), 4),
 (('anchor', 'hanov', 'group'), 3),
 (('data', 'govern', 'refer'), 3),
 (('govern', 'refer', 'master'), 3),
 (('refer', 'master', 'data'), 3),
 (('master', 'data', 'busi'), 3),
 (('heineken', 'netherland', 'commerc'), 3),
 (('heineken', 'nethe

In [31]:
tokens_sorted

[('attest', 22),
 ('di', 17),
 ('game', 17),
 ('aso', 12),
 ('fitch', 12),
 ('blackrock', 12),
 ('bbc', 12),
 ('rocco', 12),
 ('var', 12),
 ('izettl', 12),
 ('criteo', 11),
 ('marsh', 11),
 ('biobank', 11),
 ('virgin', 11),
 ('nous', 11),
 ('client', 10),
 ('leadership', 10),
 ('ubisoft', 10),
 ('metapack', 10),
 ('bloomberg', 10),
 ('roger', 10),
 ('gobiern', 10),
 ('rsquo', 10),
 ('et', 10),
 ('porsch', 10),
 ('quizlet', 10),
 ('airbus', 10),
 ('nexthink', 10),
 ('dati', 9),
 ('h', 9),
 ('trivago', 9),
 ('vmware', 9),
 ('heuritech', 9),
 ('customer', 9),
 ('atos', 9),
 ('bof', 8),
 ('mail', 8),
 ('savil', 8),
 ('jeux', 8),
 ('hungari', 8),
 ('rabobank', 8),
 ('stfs', 8),
 ('stda', 8),
 ('aalbert', 8),
 ('skello', 8),
 ('beazley', 8),
 ('aviva', 8),
 ('procedur', 7),
 ('guru', 7),
 ('stress', 7),
 ('invest', 7),
 ('bnp', 7),
 ('pariba', 7),
 ('civil', 7),
 ('équip', 7),
 ('element', 7),
 ('ellipt', 7),
 ('jam', 7),
 ('chef', 7),
 ('mclaren', 7),
 ('nielsen', 7),
 ('swiss', 7),
 ('trip

In [32]:
tokens_x =list(zip(*tokens_sorted))[0]
tokens_y =list(zip(*tokens_sorted))[1]

import matplotlib.pyplot as plt
#%matplotlib notebook
plt.plot(tokens_x, tokens_y);
plt.xticks(rotation=90)
plt.show()


<Figure size 640x480 with 1 Axes>

In [33]:
df.sector_final.unique()

array(["['', 'Information ', 'Technology and ', 'Services']",
       "['', 'Staffing and ', 'Recruiting', 'Consumer ', 'Goods', 'Financial ', 'Services']",
       "['', 'Marketing and ', 'Advertising', 'Computer ', 'Software', 'Internet']",
       "['', 'Information ', 'Technology and ', 'Services', 'Internet', 'Staffing and ', 'Recruiting']",
       "['', 'Information ', 'Technology and ', 'Services', 'Computer ', 'Software', 'Management ', 'Consulting']",
       "['', 'Construction', 'Staffing and ', 'Recruiting', 'Financial ', 'Services']",
       "['', 'Nonprofit ', 'Organization ', 'Management', 'Staffing and ', 'Recruiting', 'Hospital & ', 'Health ', 'Care']",
       "['', 'Computer ', 'Software', 'Information ', 'Technology and ', 'Services']",
       "['', 'Electrical/', 'Electronic ', 'Manufacturing', 'Marketing and ', 'Advertising', 'Biotechnology']",
       "['', 'Information ', 'Technology and ', 'Services', 'Computer ', 'Software', 'Financial ', 'Services']",
       "['', 

In [34]:
p = ['!', '(', ')', '-', '[', ']', '{', '}', ';', ':', "'", '"', '\\', ',', '<', '>', '.', '/', '?', '@', '#', '$', '%', '^', '&', '*', '_', '~']

In [35]:
tokens = {lst:df['role_description1'].count(lst) for lst in df['role_description1']}

tokens = {k: v for k, v in tokens.items() if v > 5}

tokens_sorted = sorted(tokens.items(), key=lambda kv: kv[1], reverse=True)

AttributeError: 'Int64Index' object has no attribute 'levels'

In [59]:
def functions_translation(functions):
    for word in functions:
        if word == 'Tecnologia da informação':
            word == 'Information Technology'
        elif word == 'Outro':
            word == 'Other'
        elif word == 'Desenvolvimento comercialVendas':
            word == 'Business DevelopmentSales'
        elif word == 'EngenhariaTecnologia da informação':
            word == 'EngineeringInformation Technology'
        else:
            return word
            

In [ ]:
functions = [word.strip() for word in df['function'] if word !='']

In [54]:
functions_dict = {word:functions.count(word) for word in functions}
functions_dict = sorted(functions_dict.items(), key=lambda kv: kv[1], reverse=True)

In [60]:
functions_translation(functions)

'Information Technology'

In [56]:
functions_dict

[('Tecnologia da informação', 239),
 ('Information Technology', 72),
 ('Outro', 31),
 ('Desenvolvimento comercialVendas', 22),
 ('Business DevelopmentSales', 12),
 ('EngenhariaTecnologia da informação', 10),
 ('Analista', 7),
 ('Tecnologia da informaçãoAnalista', 6),
 ('Other', 5),
 ('Estratégia/PlanejamentoAnalistaTecnologia da informação', 5),
 ('Marketing', 4),
 ('Financeiro', 4),
 ('Consultoria', 4),
 ('EngineeringInformation Technology', 3),
 ('PesquisaAnalista', 3),
 ('Information TechnologyAnalyst', 2),
 ('Analyst', 2),
 ('Composição e revisão de textosAnalistaFinanceiro', 2),
 ('AnalystInformation TechnologyEngineering', 2),
 ('Tecnologia da informaçãoConsultoria', 2),
 ('Tecnologia da informaçãoDesenvolvimento comercialFinanceiro', 2),
 ('Tecnologia da informaçãoEngenharia', 2),
 ('AnalistaConsultoriaTecnologia da informação', 2),
 ('Tecnologia da informaçãoAnalistaEngenharia', 2),
 ('AnalistaTecnologia da informação', 2),
 ('FinanceiroTecnologia da informação', 2),
 ('Engenha

In [37]:
df['function'][0]

'Information Technology'